In [2]:
import screen_brightness_control as sbc
import mediapipe as mp
import cv2

In [3]:
mp_hands=mp.solutions.hands
mp_draw=mp.solutions.drawing_utils
hand=mp_hands.Hands(max_num_hands=1)

In [4]:
sbc.set_brightness(100)

In [5]:
import numpy as np

In [12]:
def distance(landmarks,frame):
    h,w,_=frame.shape
    x1=int(landmarks.landmark[8].x*w)
    y1=int(landmarks.landmark[8].y*h)
    z1=int(landmarks.landmark[8].z)
    x2=int(landmarks.landmark[4].x*w)
    y2=int(landmarks.landmark[4].y*h)
    z2=int(landmarks.landmark[4].z)
    d = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + 0.5)
    return np.interp(d, [30, 300], [0, 100]),((x1,y1),(x2,y2))


In [20]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if not ret:
        print("nothing")
        break
    h,w,_=frame.shape
    frame_rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    result=hand.process(frame_rgb)
    if result.multi_hand_landmarks:
        landmarks=result.multi_hand_landmarks[0]
        brightness,points=distance(landmarks,frame)
        point1 = (points[0][0], points[0][1])  
        point2 = (points[1][0], points[1][1]) 
        cv2.circle(frame, point1, 10, (0, 0, 255), cv2.FILLED)  # Red circle at index fingertip
        cv2.circle(frame, point2, 10, (0, 255, 0), cv2.FILLED)  # Green circle at thumb tip

        sbc.set_brightness(brightness)
        cv2.putText(frame, f"BRIGHTNESS: {int(brightness)}", (10, h - 20),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        
    cv2.imshow("frame",frame)
    if cv2.waitKey(1)==ord("q"):
        cap.release()
        cv2.destroyAllWindows()
        break   

